In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.random.set_seed(777)

In [5]:
x_train = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]

y_train = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# Evaluation our model using this test dataset
x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

In [15]:
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).batch(len(x_train))

In [8]:
W = tf.Variable(tf.random.normal((3, 3)))
b = tf.Variable(tf.random.normal((3,)))

In [23]:
def softmax_fn(features):
    # softmax를 가설로 사용
    hypothesis = tf.nn.softmax(tf.matmul(features, W) + b)
    return hypothesis

In [24]:
def loss_fn(hypothesis, features, labels):
    # 비용 함수는 Cross-entropy (-sigma(Li * log(Si)))
    cost = tf.reduce_mean(-tf.reduce_sum(
        labels * tf.math.log(hypothesis), axis=1))
    return cost

In [21]:
is_decay = True
starter_learning_rate = 0.1

# 여기가 이번 강의에서 중요한 부분
if(is_decay):
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        # starter_learning_rate : 0.1로 설정
        initial_learning_rate=starter_learning_rate,
        # epoch 1000번 마다 decay 해줌
        decay_steps=1000,
        # decay 비율은 0.96
        decay_rate=0.96,
        # 적용 유무 : decayed_learning rate =
        #     learning_rate * decay_rate ^ (global_step / decay_steps)
        staircase=True)
    optimizer = tf.keras.optimizers.SGD(learning_rate)
else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=starter_learning_rate)

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(softmax_fn(features),features,labels)
    return tape.gradient(loss_value, [W,b])

In [19]:
def accuracy_fn(hypothesis, labels):
    prediction = tf.argmax(hypothesis, 1)
    is_correct = tf.equal(prediction, tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    return accuracy

In [22]:
EPOCHS = 1001

for step in range(EPOCHS):
    for features, labels  in iter(dataset):
        # tf.cast : typecasting
        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)
        grads = grad(softmax_fn(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(
                step, loss_fn(softmax_fn(features),features,labels)))
x_test = tf.cast(x_test, tf.float32)
y_test = tf.cast(y_test, tf.float32)
test_acc = accuracy_fn(softmax_fn(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Iter: 0, Loss: 0.2263
Iter: 100, Loss: 0.2223
Iter: 200, Loss: 0.2185
Iter: 300, Loss: 0.2148
Iter: 400, Loss: 0.2112
Iter: 500, Loss: 0.2078
Iter: 600, Loss: 0.2044
Iter: 700, Loss: 0.2012
Iter: 800, Loss: 0.1981
Iter: 900, Loss: 0.1951
Iter: 1000, Loss: 0.1922
Testset Accuracy: 1.0000
